In [ ]:
!pip install pymessenger

In [ ]:
#需要用的模組
from flask import Flask, request
#定義APP
app=Flask(__name__)
#定義第一種route網址與模式
@app.route("/", methods=['GET'])
def aa():
    print("登錄成功並在網址增加此/user/ID=<user_id>&message=<message>")
    return "登錄OK"#以字串型式在瀏覽器顯示
#定義第二種route網址與模式
@app.route("/user/ID=<user_id>&message=<message>", methods=['GET'])
def bb(user_id,message):
    UID={}
    UID[user_id]=message
    print(UID)
    STR="你的使用者ID:"+user_id+"\n你的留言:"+message+"
    return STR#以字串型式在瀏覽器顯示
if __name__ == "__main__":
    app.run(port=1216)

In [ ]:
https://65ec-2402-7500-92d-1f2-a1ca-754f-a5b9-6d80.ngrok-free.app/verify

In [ ]:
import os, sys
import json
from flask import Flask, request
from pymessenger import Bot
#https://github.com/davidchua/pymessenger
app = Flask(__name__)
PAGE_ACCESS_TOKEN = "EAANYcH5lQZCMBAHU1tDhgO7bmkBgQtJD83vT9ZAAlKnAQEHurqUrYOZC8p9qna3fZCfsbNCsxucJ8ifmBkIlFV4ludX3XV53ehB0umvhuojpK0OuEBNZBT7On6nbMCZBHNYt7u6coPgBqGZBQ4pA2PnWOANccWXhcmhHe1et4psyb820ArSAaHL"
bot = Bot(PAGE_ACCESS_TOKEN)
num1={"A":"營業時間","B":"連絡電話","C":"營業地址"}
str1="可以輸入以下代碼查詢資料A:營業時間,B:連絡電話,C:營業地址"
@app.route('/', methods=['GET'])
def test():
    return "沒看到沒東西嗎,還在學啦", 200
@app.route('/verify', methods=['GET'])
def verify():
 # Webhook verification
    print(request.args)
    if request.args.get("hub.mode") == "subscribe" and request.args.get("hub.challenge"):
        if not request.args.get("hub.verify_token") == "1234":
            return "Verification token mismatch", 403
        return request.args["hub.challenge"], 200
    return "放下屠刀,拿起AK", 200
@app.route('/verify', methods=['POST'])
def webhook():
    data = request.get_json()
    log(data)
    if data['object'] == 'page':
        for entry in data['entry']:
            for messaging_event in entry['messaging']:
                # IDs
                sender_id = messaging_event['sender']['id']
                recipient_id = messaging_event['recipient']['id']
                if 'message' in messaging_event:
                     # Extracting text message
                    if 'text' in messaging_event['message']:
                        text=messaging_event['message']['text']
                        print("ID:%s,Text:%s"%(messaging_event['sender']['id'],text))
                        if text=="A" or text=="B" or text=="C":
                            bot.send_text_message(sender_id,num1[text])
                        else:
                            bot.send_text_message(sender_id,str1)
                return "ok", 200
def log(message):
    print(message)
    sys.stdout.flush()
if __name__ == "__main__":
    app.run(port=1216)

In [ ]:
#猜數字
import os, sys
import json
from flask import Flask, request
from pymessenger import Bot
#https://github.com/davidchua/pymessenger
import random as rd
app = Flask(__name__)
PAGE_ACCESS_TOKEN = "EAANYcH5lQZCMBAHU1tDhgO7bmkBgQtJD83vT9ZAAlKnAQEHurqUrYOZC8p9qna3fZCfsbNCsxucJ8ifmBkIlFV4ludX3XV53ehB0umvhuojpK0OuEBNZBT7On6nbMCZBHNYt7u6coPgBqGZBQ4pA2PnWOANccWXhcmhHe1et4psyb820ArSAaHL"
bot = Bot(PAGE_ACCESS_TOKEN)
user={}
@app.route('/verify', methods=['GET'])
def verify():
 # Webhook verification
    print(request.args)
    if request.args.get("hub.mode") == "subscribe" and request.args.get("hub.challenge"):
        if not request.args.get("hub.verify_token") == "1234":
            return "Verification token mismatch", 403
        return request.args["hub.challenge"], 200
    return "不正確的地方，請出去", 200
@app.route('/verify', methods=['POST'])
def webhook():
    data=request.get_json()
    log(data)
    UID=data['entry'][0]['messaging'][0]['sender']['id']
    TEXT=data['entry'][0]['messaging'][0]['message']['text']
    #初始化數值
    if UID not in user:
        user[UID]=[rd.randint(1,100),1,100,"Y"]
    elif user[UID][3]=="N":
        user[UID]=[rd.randint(1,100),1,100,"Y"]
    #開始判斷
    guess=int(TEXT)
    if user[UID][0]==guess:
        bot.send_text_message(UID,"恭喜猜對")
        user[UID][3]="N"
    elif user[UID][0]>guess:
        user[UID][1]=guess
        bot.send_text_message(UID,"請輸入%d到%d的數字"%(user[UID][1],user[UID][2]))
    else:
        user[UID][2]=guess
        bot.send_text_message(UID,"請輸入%d到%d的數字"%(user[UID][1],user[UID][2]))
    return "ok", 200
def log(message):
    print(message)
    sys.stdout.flush()
if __name__ == "__main__":
    app.run(debug = 0, port = 1216)